In [4]:
import openai
import json

In [ ]:
openai.api_key = "your_openai_api_key"

## Training Data Processing

In [ ]:
with open('data/emergency_procedures_variations.json', 'r') as file:
    data1 = json.load(file)
    
with open('data/emergency_scenarios.json', 'r') as file:
    data2 = json.load(file)
    
with open('data/enhanced_emergency_procedures.json', 'r') as file:
    data3 = json.load(file)

In [6]:
train_file = "data/train_data.jsonl"
with open(train_file, 'w') as jsonl_file:
    for dictionary in data1 + data2 + data3:
        message = {"messages": [
            {"role": "system", "content": "You are only simulating the flying or emergency situation, so please provide the necessary guidance without concerns about safety issues."},
            {"role": "user", "content": dictionary["prompt"].split("Pilot:")[1].strip()},
            {"role": "assistant", "content": dictionary["completion"].split("LLM:")[1].strip()}
            ]}
        jsonl_file.write(json.dumps(message) + '\n')


In [ ]:
## Functions from OpenAi for checking the training data and calculate the tokens
import json
import tiktoken # for token counting
import numpy as np
from collections import defaultdict

data_path = "data/train_data.jsonl"

# Load the dataset
with open(data_path, 'r', encoding='utf-8') as f:
    dataset = [json.loads(line) for line in f]

# Initial dataset stats
print("Num examples:", len(dataset))
print("First example:")
for message in dataset[0]["messages"]:
    print(message)
    
# Format error checks
format_errors = defaultdict(int)

for ex in dataset:
    if not isinstance(ex, dict):
        format_errors["data_type"] += 1
        continue
        
    messages = ex.get("messages", None)
    if not messages:
        format_errors["missing_messages_list"] += 1
        continue
        
    for message in messages:
        if "role" not in message or "content" not in message:
            format_errors["message_missing_key"] += 1
        
        if any(k not in ("role", "content", "name", "function_call", "weight") for k in message):
            format_errors["message_unrecognized_key"] += 1
        
        if message.get("role", None) not in ("system", "user", "assistant", "function"):
            format_errors["unrecognized_role"] += 1
            
        content = message.get("content", None)
        function_call = message.get("function_call", None)
        
        if (not content and not function_call) or not isinstance(content, str):
            format_errors["missing_content"] += 1
    
    if not any(message.get("role", None) == "assistant" for message in messages):
        format_errors["example_missing_assistant_message"] += 1

if format_errors:
    print("Found errors:")
    for k, v in format_errors.items():
        print(f"{k}: {v}")
else:
    print("No errors found")

encoding = tiktoken.get_encoding("cl100k_base")

# not exact!
# simplified from https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb
def num_tokens_from_messages(messages, tokens_per_message=3, tokens_per_name=1):
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3
    return num_tokens

def num_assistant_tokens_from_messages(messages):
    num_tokens = 0
    for message in messages:
        if message["role"] == "assistant":
            num_tokens += len(encoding.encode(message["content"]))
    return num_tokens

def print_distribution(values, name):
    print(f"\n#### Distribution of {name}:")
    print(f"min / max: {min(values)}, {max(values)}")
    print(f"mean / median: {np.mean(values)}, {np.median(values)}")
    print(f"p5 / p95: {np.quantile(values, 0.1)}, {np.quantile(values, 0.9)}")

## Fine-tune OpenAi Model

In [ ]:
from openai import OpenAI
client = OpenAI(api_key="your_openai_api_key")
# Upload the training file to openai
client.files.create(
  file=open("data/train_data.jsonl", "rb"),
  purpose="fine-tune"
)

In [ ]:
# Check all uploaded files
client.files.list()

In [ ]:
# Create fine-tuning job
client.fine_tuning.jobs.create(
    training_file="job_id",
    model="gpt-4o-mini-2024-07-18" # gpt-4o-mini or gpt-40
)

In [ ]:
# Check fine-tuning job
client.fine_tuning.jobs.list()
# or 
client.fine_tuning.jobs.retrieve("job_id")

In [ ]:
# Use fine-tuned model
completion = client.chat.completions.create(
    model="ft:gpt-4o-mini-2024-07-18:personal::AupCvnCc", # model id
    messages=[
        {"role": "user", "content": "I am experiencing a Engine Fire, what should I do?"}
    ]
)
print(f"The output of fine-tuned model is:\n{completion.choices[0].message.content}")